In [ ]:
# Interactive Radial Visualization for 470x470 Distance Matrix
# Paste this into a Jupyter Notebook cell and run.
# Make sure you:  pip install ipympl  (then restart kernel)

# https://chatgpt.com/share/687abc0d-b634-8013-9fe3-d35e8421665f



# %matplotlib widget

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

# --- Load the distance matrix ---
# (Assumes CSV has a header row/col with sample labels)
df = pd.read_csv('D:\Github\phone-cleaner\Phonetic_Distance\charts-simple\distance.csv', index_col=0)

# --- Create the selection widget ---
sample_names = list(df.index)
listbox = widgets.Select(
    options=sample_names,
    description="Sample:",
    rows=15  # shows 15 items at a time; scroll for the rest
)

# --- Initialize the plot ---
fig, ax = plt.subplots(figsize=(6, 6))
ax.set_aspect('equal')
ax.set_xticks([]); ax.set_yticks([])
ax.set_title("Select a sample to view its 40 nearest neighbors")
fig.canvas.draw()

# --- Update function ---
def update_plot(selected_item):
    ax.clear()
    ax.set_aspect('equal')
    ax.set_xticks([]); ax.set_yticks([])

    # Get distances and 40 nearest neighbors (excluding self)
    distances = df.loc[selected_item]
    nearest40 = distances[distances > 0].nsmallest(40)

    # Compute polar coordinates (angle evenly spaced clockwise, radius = distance)
    N = len(nearest40)
    angles = [-2 * np.pi * (i / N) for i in range(N)]
    radii = nearest40.values

    xs = radii * np.cos(angles)
    ys = radii * np.sin(angles)

    # Scale the axes so all points fit nicely
    r_max = radii.max() if N > 0 else 1.0
    lim = max(3.0, r_max * 1.2 + 0.5)
    ax.set_xlim(-lim, lim)
    ax.set_ylim(-lim, lim)

    # Draw center point
    ax.scatter(0, 0, color='red', s=40, zorder=3)
    ax.text(0, 0, selected_item, color='red', fontsize=9, ha='right', va='bottom')

    # Draw neighbors
    ax.scatter(xs, ys, color='blue', s=20, zorder=2)
    for label, x, y in zip(nearest40.index, xs, ys):
        ax.text(x, y, label, fontsize=8, ha='center', va='center')

    ax.set_title(f"Closest 40 items to '{selected_item}'")
    fig.canvas.draw_idle()

# --- Link widget to plot update ---
def on_selection_change(change):
    if change['name'] == 'value' and change['new'] is not None:
        update_plot(change['new'])

listbox.observe(on_selection_change, names='value')

# --- Display the interface ---
ui = widgets.HBox([listbox, fig.canvas])
display(ui)


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/distance.csv'